# Bi-directional LSTM Named Entity Tagger

Named-entity recognition (NER) (also known as entity identification, entity chunking and entity extraction) is a subtask of information extraction that seeks to locate and classify named entities in text into pre-defined categories such as the names of persons, organizations, locations, expressions of times, quantities, monetary values, percentages, etc.

Most research on NER systems has been structured as taking an unannotated block of text, such as the following **example**:

**INPUT:** Jim bought 300 shares of Acme Corp. in 2006.

And producing an annotated block of text that highlights the names of entities:

**OUTPUT:** [Jim]Person bought 300 shares of [Acme Corp.]Organization in [2006]Time.

In this example, a person name consisting of one token, a two-token company name and a temporal expression have been detected and classified.(Wikipedia)

Your task in this lab is to implement named entity bi-LSTM based tagger which uses a bi-directional LSTM to extract features from the input sentence, which are then passed through a multi-layer perceptron to predict
the tag of the word. Finally, train that model on [WikiNER](https://github.com/neulab/dynet-benchmark/tree/master/data/tags) dataset.

In [1]:
# Set-Up related files and Hyper-parameters
using Knet
Pkg.installed("ArgParse") == nothing && Pkg.add("ArgParse")
using ArgParse
include(Pkg.dir("Knet","data","wikiner.jl"));

## Problem 1. Implement Minibatching
Your first task is to implement minibatching. Remember the purpose of minibatching is to feed model with multiple instances in parallel. For example, suppose you are given 2 sentences with a batchsize of 2:<br/>
```julia
julia> show_batch() # show batch in not implemented in Knet, it is a custom function
Inputs sentences:
Sent-> That inscribed in the genealogical records of his family is Jiang Zhoutai .
NERs-> O    O         O  O   O            O       O  O   O      O  I-PER I-PER   O
Sent-> Wallace was a prolific author .
NERs-> I-PER   O   O O        O      O

Minibatched inputs:
Time step 1 ---> Inputs: That  Wallace 
                 Outputs: O     I-PER
Time step 2 ---> Inputs: inscribed  was 
                 Outputs:O          O
Time step 3 ---> Inputs: in  a 
                 Outputs: O  O
Time step 4 ---> Inputs: the  prolific 
                 Outputs: O  O
Time step 5 ---> Inputs: genealogical  author 
                 Outputs: O              O
Time step 6 ---> Inputs: records  . 
                 Outputs: O       O
Time step 7 ---> Inputs: of 
                 Outputs: O
Time step 8 ---> Inputs: his 
                 Outputs: O
Time step 9 ---> Inputs: family 
                 Outputs: O
Time step 10 --->Inputs: is 
                 Outputs: O
Time step 11 ---> Inputs: Jiang 
                  Outputs: I-PER
Time step 12 ---> Inputs: Zhoutai 
                  Outputs: I-PER
Time step 13 ---> Inputs: . 
                  Outputs: O
Batchsizes
2 2 2 2 2 2 1 1 1 1 1 1 1
```
As you see from the previous code snippet, at each time you need to feed your model with **possible** maximum number of words, e.g. at most 2 at least 1 for these 2 sentences.

In [2]:
# You are given make_batches function
function make_batches(data, w2i, t2i, batchsize)
    batches = []
    sorted = sort(data, by=length, rev=true)
    for k = 1:batchsize:length(sorted)
        lo = k
        hi = min(k+batchsize-1, length(sorted))
        samples = sorted[lo:hi]
        batch = make_batch(samples, w2i, t2i)
        push!(batches, batch)
    end
    return batches
end
#  You need to implement make_batch function
function make_batch(samples, w2i, t2i)
    input = Int[]
    output = Int[]
    longest = length(samples[1])
    batchsizes = zeros(Int, longest)
    # YOUR ANSWER
    for i in 1:longest
        batch=0
        for j in 1:length(samples)
            lengthSentence=length(samples[j])
            if lengthSentence>=i
                indx=get(w2i,samples[j][i][1],w2i[UNK])
                push!(input,indx)
                indx=get(t2i,samples[j][i][2],t2i["O"])
                push!(output,indx)
                batch+=1
            end 
        end
        batchsizes[i]=batch
    end
    return input, output, batchsizes
end

make_batch (generic function with 1 method)

In [3]:
# w[1]   => weight/bias params for forward LSTM network
# w[2:5] => weight/bias params for MLP+softmax network
# w[6]   => word embeddings
# w[7]   => rnnstruct given by rnninit function
# Hint you mave take a look main function below to better understand its calling convention
function initweights(atype, hidden, words, tags, embed, mlp, usegpu, winit=0.01)
    w = Array{Any}(7)
    # YOUR ANSWER
    w[6]=atype(randn(embed,words).*winit)
    w[7],w[1] = rnninit(embed,hidden; rnnType=:lstm , bidirectional=true)
    w[2]=atype(randn(mlp,2*hidden).*winit)
    w[3]=atype(zeros(mlp,1))
    w[4]=atype(randn(tags,mlp).*winit)
    w[5]=atype(zeros(tags,1)) 
    #println("embed", embed)
    #println("hidden", hidden)
    #println("words ", words)
    #println("size of w1 ", size(w[1]))
     return w
end

initweights (generic function with 2 methods)

## Problem 3. Implement Predict function
Returns scores for output tags.

In [4]:
function predict(ws, xs, batchsizes)
    # YOUR ANSWER
    w_r,w_mlp,b_mlp,w_y,b_y,wx,r = ws
    x = wx[:,xs] # xs is embedding that we init randomly, we are finding the x that corresponds the one column in embedd
    (y,_ )= rnnforw(r,w_r,x,batchSizes=batchsizes) # y=(H,ΣBt)
    y=sigm.(w_mlp*y.+b_mlp)    
    return w_y * y .+ b_y  # return=(V,ΣBt)
end

predict (generic function with 1 method)

## Problem 4. Implement Loss function
That function basically takes the predictions and returns the negative log-likelihood of these predictions as loss.
Hint: You may have a look Knet's ```nll``` function

In [5]:
# our loss function
function loss(w, x, ygold, batchsizes)
    return nll(predict(w,x,batchsizes),ygold)
end

loss (generic function with 1 method)

In [6]:
lossgradient = gradloss(loss) # Knet's automatic gradient calculator function

(::gradfun) (generic function with 1 method)

## Problem 5. Implemlent Train function
That function takes the model(w), input(x) and correct labels(ygold)
and updates the model(w) parameters.
**Hints**
Remember lossgradient function returns the loss value and gradients as 2 arguments:
```
gradients_of_loss, lossval = lossgradient(.....)
```

In [7]:
function train!(w, x, ygold, batchsizes, opt)
    # YOUR ANSWER
    gradients_of_loss, lossval=lossgradient(w,x,ygold,batchsizes)
    update!(w,gradients_of_loss,opt)
   
    return lossval
end

train! (generic function with 1 method)

## Problem 6. Implement Accuracy function
Accuracy function counts the number of words(tokens) and also counts the number of correctly predicted tokens and returns ```number_of_correctly_pred_token / number_of_total_token```

In [8]:
function tagval(a)
    m = maximum(a,1)
    b=Array{Any}(size(a,2))
    for j in 1:size(a,2)
        for i in 1:size(a,1)
            if m[j]==a[i,j]
                b[j] = i
            end 
        end
    end
    return b
end

tagval (generic function with 1 method)

In [13]:
function accuracy(w, batches, i2t)
    # YOUR ANSWER
    correct_num=0
    token_num=0
    for (k,batch) in enumerate(batches)
        x,ygold,batchSizes=batch 
        token_num+= sum(batchSizes)

        preds=predict(w,x,batchSizes)
        preds=convert(Array,preds) 
        maxval = maximum(preds,1)
        ypred=tagval(preds)
        for i in 1:length(ygold)
            if(ygold[i]==ypred[i])
                correct_num+=1
            end
        end
    end
    tag_acc=correct_num/token_num   
    return tag_acc
end


accuracy (generic function with 1 method)

In [11]:
# Do not touch this function
function main(args)
    s = ArgParseSettings()
    s.description = "Bidirectional LSTM Tagger in Knet"

    @add_arg_table s begin
        ("--usegpu"; action=:store_true; help="use GPU or not")
        ("--embed"; arg_type=Int; default=128; help="word embedding size")
        ("--hidden"; arg_type=Int; default=50; help="LSTM hidden size")
        ("--mlp"; arg_type=Int; default=32; help="MLP size")
        ("--epochs"; arg_type=Int; default=20; help="number of training epochs")
        ("--minoccur"; arg_type=Int; default=6; help="word min occurence limit")
        ("--report"; arg_type=Int; default=500; help="report period in iters")
        ("--valid"; arg_type=Int; default=10000; help="valid period in iters")
        ("--seed"; arg_type=Int; default=-1; help="random seed")
        ("--batchsize"; arg_type=Int; default=100; help="batchsize")
    end

    isa(args, AbstractString) && (args=split(args))
    o = parse_args(args, s; as_symbols=true)
    o[:seed] > 0 && Knet.setseed(o[:seed])
    atype = o[:atype] = !o[:usegpu] ? Array{Float32} : KnetArray{Float32}
    #datadir = abspath(joinpath(@__DIR__, "../data/tags"))
    datadir = WIKINER_DIR

    # load WikiNER data
    data = WikiNERData(datadir, o[:minoccur])

    # build model
    nwords, ntags = length(data.w2i), data.ntags
    w = initweights(
        atype, o[:hidden], nwords, ntags,  o[:embed], o[:mlp], o[:usegpu])
    opt = optimizers(w, Adam)

    # make batches
    trn = make_batches(data.trn, data.w2i, data.t2i, o[:batchsize])
    dev = make_batches(data.dev, data.w2i, data.t2i, o[:batchsize])

    # train bilstm tagger
    nwords = data.nwords
    println("nwords=$nwords, ntags=$ntags"); flush(STDOUT)
    println("startup time: ", Int((now()-t00).value)*0.001); flush(STDOUT)
    t0 = now()
    all_time = dev_time = all_tagged = this_tagged = this_loss = 0
    for epoch = 1:o[:epochs]
        # training
        shuffle!(trn)
        for (k,batch) in enumerate(trn)
            x, ygold, batchsizes = batch
            num_tokens = sum(batchsizes)
            batch_loss = train!(w, x, ygold,  batchsizes, opt)
            this_loss += num_tokens*batch_loss
            this_tagged += num_tokens
        end

        # validation
        dev_start = now()
        tag_acc = accuracy(w, dev, data.i2t)
        dev_time += Int((now()-dev_start).value)*0.001
        train_time = Int((now()-t0).value)*0.001-dev_time

        # report
        @printf("epoch %d finished, loss=%f\n", epoch, this_loss/this_tagged)
        all_tagged += this_tagged
        this_loss = this_tagged = 0
        all_time = Int((now()-t0).value)*0.001
        @printf("tag_acc=%.4f, time=%.4f, word_per_sec=%.4f\n",
                tag_acc, train_time, all_tagged/train_time)
        flush(STDOUT)
    end
end

main (generic function with 1 method)

In [ ]:
t00 = now();main("--usegpu")

nwords=119101, ntags=9
startup time: 15.527000000000001
epoch 1 finished, loss=0.418017
tag_acc=0.8485, time=84.7930, word_per_sec=41272.3456
epoch 2 finished, loss=0.222018
tag_acc=0.9043, time=163.8070, word_per_sec=42728.4060
epoch 3 finished, loss=0.130626
tag_acc=0.9258, time=247.6100, word_per_sec=42400.6219
epoch 4 finished, loss=0.093117
tag_acc=0.9314, time=333.6950, word_per_sec=41949.7565
epoch 5 finished, loss=0.075671
tag_acc=0.9327, time=421.1340, word_per_sec=41549.7918
epoch 6 finished, loss=0.064969
tag_acc=0.9296, time=501.9500, word_per_sec=41832.1267
epoch 7 finished, loss=0.056741
tag_acc=0.9302, time=593.2460, word_per_sec=41293.5646
epoch 8 finished, loss=0.049628
tag_acc=0.9296, time=679.6440, word_per_sec=41193.4013
epoch 9 finished, loss=0.043440
tag_acc=0.9263, time=762.2220, word_per_sec=41321.8905
